In [1]:
from excelparser import ExcelAdapter
import langchain
import yaml
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
# from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
import pinecone
import os

from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
import loguru
import asyncio

In [3]:
with open("../CREDS/keys.yaml", "r") as file:
    configs = yaml.safe_load(file)

In [5]:
# configs

In [7]:
def get_raw_files_from_source(path: str) -> list[str]:
    return [file for file in os.listdir(path) if file not in ['.DS_Store']]

In [21]:
def parse_raw_files(path_to_raw_file_directory: str) -> list[dict]:
    files = get_raw_files_from_source(path_to_raw_file_directory)
    knowledge_base = []
    for file in files:
        ext = file.split('.')[-1]
        print(f"Current file format: {ext}")
        if ext in ['xls', 'xlsx']:
            with open(f"{path_to_raw_file_directory}{file}", "rb") as f:
                file_bytes = f.read()
            result = ExcelAdapter().parse(file_bytes, {'file_name': file})
            knowledge_base.append(result)
            continue
        else:
            print(f"File format {ext} is not currently supported for parsing.")
            continue
    return knowledge_base

In [23]:
get_raw_files_from_source("./RAWDATA/")

['NASDAQ-SAIA.xlsx']

In [25]:
parse_raw_files("./RAWDATA/")

Current file format: xlsx
ExcelAdapter instance created.
Parsing file with metadata {}


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/reader/excel.py:237: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()


[{'content': "              Stock Price Chart for past 4 Years: Risk Free Rate for past 4 Years:\nCOMPANY NAME:   SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc.  IR WEBPAGE: SAIA IR Page Link SAIA IR Page Link  \nTICKER   SAIA SAIA   FY ENDS IN: 2025-12-31 00:00:00 2025-12-31 00:00:00  $ IN: $000,000 $000,000  \nSAIA DCF      SAIA MARKET     PRICE    \nSHARE VALUE:   520.1066763818468 520.1066763818468  SHARE PRICE:  455.73 455.73 Base DIFFERENCE (%):  0.1412605630128514  \nSAIA DCF      SAIA MARKET      MODEL   \nSHARE VALUE DATE:   2024-12-31 00:00:00 2024-12-31 00:00:00  SHARE PRICE DATE:   2024-12-31 00:00:00 2024-12-31 00:00:00  BASE YEAR: 2023  \nCREATED BY:   Mihir Patel            \nPhone:   +1 (251) 321 5980            \nEmail:   pate1392@umn.edu            \n 2020A 2021A 2022A 2023A 2024E 2025E 2026E 2027E 2028E      \n913\nSAIA, Inc.           Model Case: Base SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc. SAIA, Inc

In [ ]:
def main():
    files =